<a href="https://colab.research.google.com/github/rtajeong/ChatGPT_for_Management/blob/main/3_gpt_direct_or_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT API 활용
- GPT API 활용의 효율성을 보여주기 위한 예제임.

# 실습 주제: 고객 리뷰 데이터 분석 및 요약
- 목표:
  - 스타트업 경쟁 시장을 분석하는 과제를 통해 데이터 수집, 전처리, 요약, 시각화, 보고서 작성의 전 과정을 경험.
- 배경 시나리오:
  - 한 스타트업이 투자 유치를 준비 중인데 경쟁 업체들의 정보를 분석해 보고서를 작성하려고 한다. 이를 위해 데이터 수집부터 보고서 작성까지 GPT API를 활용하고자 한다.
- 문제 해결 흐름
  - 1. 데이터 준비
  - 2. 단계별 작업
    - 데이터 탐색 및 분석: GPT를 사용해 각 업체의 특징을 분석하고 주요 경쟁사를 식별.
    - 텍스트 데이터 요약: GPT로 Description 열을 요약해 주요 키워드를 추출.
    - 재무 및 직원 분석: GPT를 사용해 매출 대비 직원 수의 효율성을 분석.
    - 시장 트렌드 작성: GPT를 사용해 시장 트렌드를 요약하고 분석.
    - 최종 보고서 작성: GPT를 사용해 모든 분석 결과를 종합한 보고서를 생성.

## 데이터 파일

In [ ]:
%%writefile startup_data.csv
Company,Revenue (M USD),Employees,Founded,Description
TechFlow,25,50,2015,Specializes in AI-based chatbots for customer service.
GreenEdge,40,70,2013,Focuses on developing renewable energy technologies.
CloudSphere,60,100,2010,Offers scalable cloud storage and data solutions.
HealthBridge,15,40,2018,Develops wearable health monitoring devices.

Overwriting startup_data.csv


## GPT 대화를 통한 해결 (예시 프롬프트)
- 각 단계를 GPT 와의 개별 대화를 통해 준비한다.

```
1. 회사별 비지니스 모델 및 경쟁력 요약
'다음 파일에는 스타트업의 정보가 포함되어 있습니다. 각
회사의 설명을 기반으로 비즈니스 모델과 시장에서의 경쟁력을
한 문장으로 요약해 주세요. 파일이름:startup_data.csv
각 회사의 비즈니스 전략과 시장 경쟁력을 분석해 주세요.'

2. 비즈니스 모델의 강점 및 약점 분석

'각 회사들의 비즈니스 모델에서 강점과 약점을 간략히 작성해 주세요.'

3. 시장 진입 전략 제안
'다음 회사들의 비즈니스 모델을 바탕으로, 각 회사가 시장에서
성공하기 위해 어떤 전략을 수립해야 할지 제안해 주세요.'

4. 보고서 작성
`위 내용을 담고 있는 파일을 만들어 주세요.`
```

## openai API를 이용한 솔루션

In [ ]:
from google.colab import drive
import openai
import os

drive.mount('/content/drive')

api_key_file = '/content/drive/My Drive/Colab Notebooks/api_key.txt'

with open(api_key_file, 'r') as f:
    api_key = f.read().strip()

os.environ['OPENAI_API_KEY'] = api_key

Mounted at /content/drive


In [ ]:
from openai import OpenAI
import pandas as pd

# OpenAI 클라이언트 초기화
client = OpenAI()

# GPT Chat 호출 함수
def gpt_chat(messages, max_tokens=200):
    response = client.chat.completions.create(
        # model="gpt-4",
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

# 데이터 불러오기
df = pd.read_csv("startup_data.csv")

# 1. 회사 설명 요약
def summarize_description(description):
    messages = [
        {"role": "system", "content": "You are a business analyst specializing in concise and professional summaries."},
        {"role": "user", "content": f"Summarize the following company's description in one sentence:\n\n{description}"}
    ]
    return gpt_chat(messages)

df['Summary'] = df['Description'].apply(summarize_description)

# 2. 매출 대비 직원 효율성 분석
def analyze_efficiency(row):
    revenue = row['Revenue (M USD)']
    employees = row['Employees']
    messages = [
        {"role": "system", "content": "You are a business consultant analyzing organizational efficiency."},
        {"role": "user", "content": (
            f"Analyze the efficiency of a company with the following data:\n"
            f"- Revenue: {revenue}M USD\n"
            f"- Employees: {employees}\n\n"
            "Provide insights into their efficiency compared to industry standards and suggest improvements."
        )}
    ]
    return gpt_chat(messages)

df['Efficiency_Analysis'] = df.apply(analyze_efficiency, axis=1)

# 3. 시장 트렌드 분석
def analyze_market_trends(data):
    messages = [
        {"role": "system", "content": "You are a market analyst identifying trends and opportunities."},
        {"role": "user", "content": (
            "Based on the following company data, analyze the current market trends and opportunities:\n\n"
            f"{data}\n\n"
            "Provide insights into emerging sectors, areas of intense competition, and potential niches."
        )}
    ]
    return gpt_chat(messages, max_tokens=300)

market_trends = analyze_market_trends(df[['Company', 'Description']].to_string())

# 4. 보고서 생성
def generate_report(summaries, trends):
    messages = [
        {"role": "system", "content": "You are a professional report writer specializing in competitive analysis."},
        {"role": "user", "content": (
            f"Generate a detailed report based on the following analysis:\n\n"
            f"Company Summaries:\n{summaries}\n\n"
            f"Market Trends:\n{trends}\n\n"
            "Structure the report with the following sections:\n"
            "1. Executive Summary\n"
            "2. Detailed Competitor Analysis\n"
            "3. Market Trends and Insights\n"
            "4. Strategic Recommendations\n\n"
            "Ensure the tone is professional and the insights actionable."
        )}
    ]
    return gpt_chat(messages, max_tokens=500)

report = generate_report(
    summaries=df[['Company', 'Summary']].to_string(),
    trends=market_trends
)

# 결과 저장
df.to_csv("startup_analysis.csv", index=False)
with open("startup_report.txt", "w") as file:
    file.write(report)

print("Analysis saved to 'startup_analysis.csv'. Report saved to 'startup_report.txt'.")


Analysis saved to 'startup_analysis.csv'. Report saved to 'startup_report.txt'.


In [ ]:
df

,Company,Revenue (M USD),Employees,Founded,Description,Summary,Efficiency_Analysis
0,TechFlow,25,50,2015,Specializes in AI-based chatbots for customer ...,The company specializes in providing AI-based ...,"To analyze the efficiency of the company, we c..."
1,GreenEdge,40,70,2013,Focuses on developing renewable energy technol...,The company specializes in the development of ...,"To analyze the efficiency of the company, we c..."
2,CloudSphere,60,100,2010,Offers scalable cloud storage and data solutions.,The company provides scalable cloud storage an...,"To analyze the efficiency of the company, we c..."
3,HealthBridge,15,40,2018,Develops wearable health monitoring devices.,The company specializes in the development of ...,"To analyze the efficiency of the company, we c..."


In [ ]:
!cat startup_report.txt

## Executive Summary

This report provides a comprehensive analysis of four key companies operating in diverse sectors: TechFlow, GreenEdge, CloudSphere, and HealthBridge. Each company specializes in a unique area of technology, ranging from AI-based chatbots to renewable energy technologies. Through this analysis, we have identified emerging market trends, areas of intense competition, and potential niches for growth and innovation. Strategic recommendations are provided to capitalize on these insights and drive competitive advantage in the respective markets.

## Detailed Competitor Analysis

1. **TechFlow**: Specializing in AI-based chatbots for customer service, TechFlow leverages advanced technology to enhance customer interactions and streamline support processes. The company's focus on automation and efficiency positions it as a key player in the customer service sector.

2. **GreenEdge**: GreenEdge is dedicated to developing renewable energy technologies, capitalizing on the gr